In [2]:
import os
import tensorflow
import keras
import json

In [3]:
order_ab = json.load(open('order_ab.json'))
print(len(order_ab))
path = 'C:/users/trifo/desktop/MUMT 203 AMT/package_ab'
tie_list = []
# count = 0
for idx, dirname in enumerate(os.listdir(path)):
    # print(dirname)
    for filename in os.listdir(f'{path}/{dirname}'):
        if filename.endswith(".semantic"):
            f = open(f'{path}/{dirname}/{filename}', 'r')
            # read = f.read().split()
            total = f.read()
            if "tie" in total:
                # tie_list.append(filename.split('.')[0])
                order_ab.pop(dirname + '.wav', None)
            # print(read)
            # for j in read:
            #     if "keySignature" in j:
            #         y_sequences.add(j)
            # y_sequences.append(f.read().split())
# print(y_sequences)
print(len(order_ab))

44084
39862


In [24]:
order_aa = json.load(open('w2i_aav3.json'))
order_ab = json.load(open('w2i_abv3.json'))

In [ ]:
idx = 1407
for i in order_ab:
    if i not in order_aa:
        print(i)
        order_aa[i] = idx
        idx +=1

In [26]:
json.dump(order_aa, open('w2i_all.json', 'w'))

In [27]:
len(order_aa)

1634

In [28]:
i2w = {str(order_aa[k]): k for k in order_aa}

In [37]:
i2w['6']

'note-A#5_quarter.'

In [38]:
order_aa['note-A#5_quarter.']

6

In [39]:
json.dump(i2w, open('i2w_all.json', 'w'))

In [6]:
import copy
deep = copy.deepcopy(order_ab)

In [7]:
for i in deep:
    if order_ab[i] > 1500:
        order_ab.pop(i, None)

In [8]:
print(len(order_ab))

39559


In [22]:
order_aa = json.load(open('no_tie.json'))

In [23]:
len(order_aa)

39054

In [15]:
deep_aa = copy.deepcopy(order_aa)

In [16]:
for i in deep_aa:
    if order_aa[i] > 1500:
        order_aa.pop(i, None)

In [21]:
json.dump(order_ab, open('no_tie_ab.json', 'w'))

In [30]:
tie_list[0]

'000051661-1_1_2'

In [27]:
json_path = 'C:/users/trifo/desktop/AMT/test_order.json'
f = open(json_path)
order = json.load(f)

In [32]:
list(order)[0]

'000051650-1_1_1.wav'

In [36]:
import numpy as np

In [37]:
example = np.load('C:/Users/trifo/Desktop/AMT/batches/y/batch_1/000051653-1_2_1.npy')

In [38]:
example

array([ 959,  613,  166,  964,  350,  350,  975,  745,  169,  403,  832,
       1031, 1031,  975], dtype=int64)

In [33]:
for i in tie_list:
    order.pop(i + '.wav', None)

In [35]:
with open('C:/users/trifo/desktop/AMT/no_tie.json', 'w') as f:
    json.dump(order, f)

In [3]:
no_tie = json.load(open('C:/users/trifo/desktop/AMT/no_tie.json', 'r'))

In [46]:
# order_aa = json.load(open('no_tie_aa.json'))
order_ab = json.load(open('no_tie_ab.json'))
order_all = order_aa | order_ab
mapping = open('C:/users/trifo/desktop/AMT/w2i_all.json')
w2i = json.load(mapping)
sorted_order = list(sorted(order_ab.items(), key=lambda item: item[1], reverse=True))
max_len = 0
for img in sorted_order:
    name = img[0].split('.')[0]
    if img[0] in order_ab:
        f_in = open(f'C:/users/trifo/desktop/MUMT 203 AMT/package_ab/{name}/{name}.semantic', 'r')
    # else:
    #     f_in = open(f'C:/users/trifo/desktop/MUMT 203 AMT/package_ab/{name}/{name}.semantic', 'r')
    notes = [w2i[note] for note in f_in.read().split() if not ('barline' in note or 'keySignature' in note or 'clef' in note or 'timeSignature' in note)]
    if len(notes) > max_len:
        max_len = len(notes)
        print(max_len)
max_len

20
29
31
32
33
34
38
41
51


51

In [45]:
len(order_ab)

39559

In [1]:
import os, sys
import fnmatch
import cv2
import numpy as np
import string
import time
import json
import math

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.activations import relu, sigmoid, softmax
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers

In [3]:
class GlobalMaxPoolAcrossTime(layers.Layer):
    def __init__(self, **kwargs):
        super(GlobalMaxPoolAcrossTime, self).__init__(**kwargs)

    # (Batch_size, time, y, x, channels) -> (Batch_size, 1, y, x, channels)
    def call(self, inputs):
        shape = inputs.shape[2]
        if not inputs.shape[2]:
            shape = 64
        pixels = math.floor(shape/64)
        # pixels = int(shape/64)
        # print('pixels')
        # print('pixels',pixels)
        pool_list = []
        for i in range(0, pixels*64, pixels):
            pool_list.append(K.max(inputs[:,:,i:i+pixels], axis=2, keepdims=True))
        # return tensorflow.keras.backend.max(inputs, axis=1, keepdims=True)
        # print(len(pool_list))
        return layers.Concatenate(axis=2)(pool_list)

In [2]:
w2i = json.load(open('C:/Users/trifo/Desktop/AMT/w2i_all.json', 'r'))
len(w2i)

1634

In [3]:
# input with shape of height=32 and width=128 
inputs = Input(shape=(128,None,1))
# print(inputs.shape[0]/2)
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
print(conv_1)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2))(conv_1)
print(pool_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
print(conv_2)

pool_2 = MaxPool2D(pool_size=(2, 2))(conv_2)
print(pool_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
print(conv_3)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
print(conv_4)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 2))(conv_4)
print(pool_4)

conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
print(conv_5)

pool_5 = MaxPool2D(pool_size=(2, 1))(conv_5)
print(pool_5)

conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_5)
print(conv_6)

pool_6 = MaxPool2D(pool_size=(2, 1))(conv_6)
print(pool_5)

conv_7 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_6)
print(conv_7)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_7)
print(batch_norm_5)
 
conv_8 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
print(conv_8)
batch_norm_6 = BatchNormalization()(conv_8)
print(batch_norm_6)
pool_7 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
print(f'pool_7: {pool_7}')

conv_9 = Conv2D(512, (2,2), activation = 'relu')(pool_7)
print(f'conv_7: {conv_9}')

squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_9)
print(squeezed)
 
# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)

outputs = Dense(len(w2i)+1, activation = 'softmax')(blstm_2)

# model to be used at test time
act_model = Model(inputs, outputs)

KerasTensor(type_spec=TensorSpec(shape=(None, 128, None, 64), dtype=tf.float32, name=None), name='conv2d/Relu:0', description="created by layer 'conv2d'")
KerasTensor(type_spec=TensorSpec(shape=(None, 64, None, 64), dtype=tf.float32, name=None), name='max_pooling2d/MaxPool:0', description="created by layer 'max_pooling2d'")
KerasTensor(type_spec=TensorSpec(shape=(None, 64, None, 128), dtype=tf.float32, name=None), name='conv2d_1/Relu:0', description="created by layer 'conv2d_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32, None, 128), dtype=tf.float32, name=None), name='max_pooling2d_1/MaxPool:0', description="created by layer 'max_pooling2d_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32, None, 256), dtype=tf.float32, name=None), name='conv2d_2/Relu:0', description="created by layer 'conv2d_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 32, None, 256), dtype=tf.float32, name=None), name='conv2d_3/Relu:0', description="created by layer 'conv2d_3'")
KerasTensor(type_spec

In [5]:
def val_gen():
    path_x = 'D:/AMT/all/batches/marimba/validate_x'
    path_y = 'D:/AMT/all/batches/marimba/validate_y'
    batch_size = 16
    all_files = os.listdir(path_x)
    for idx in range(len(all_files)):
        training_img = []
        training_txt = []
        train_input_length = []
        train_label_length = []
        mini = mini = all_files[idx: idx+batch_size]
        for filename in mini:
            # Images
            img = cv2.imread(f'{path_x}/{filename}', cv2.IMREAD_GRAYSCALE)
            # img = img.reshape(img.shape[1], img.shape[0])
            # (W, H) --> (W, H, 1)
            img = np.expand_dims(img, axis=2)
            # Normalize image
            img = img / 255.
            training_img.append(img)

            # Text Targets
            text = np.load(f'{path_y}/{filename}'.split('.')[0] + '.npy')
            # if len(text) > max_label_len:
            #     max_label_len = len(text)
            training_txt.append(text)
            train_label_length.append(len(text))
            train_input_length.append(63)

        train_padded_txt = pad_sequences(training_txt, padding='post', maxlen=51)

        yield np.array(training_img), np.array(train_padded_txt), np.array(train_input_length), np.array(train_label_length)

In [8]:
val = val_gen()
act_model.load_weights('model_weights_copy.h5')
# valid_img, valid_padded_txt, valid_input_length, valid_label_length = next(val)

In [9]:
for i in range(4):
    valid_img, valid_padded_txt, valid_input_length, valid_label_length = next(val)
    prediction = act_model.predict(valid_img[:3])
    print(f"\nFirst batch | Shape {valid_img.shape[2]}\n")
    
    # use CTC decoder
    out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1], beam_width=500,
                            greedy=False)[0][0])
    i = 0
    for x in out:
        print("original_text =  ", end = '')
        for j in valid_padded_txt[i]:
            if j != 0:
                print(i2w[str(j)], end = ', ')
        print('\n')
        print("predicted text = ", end = '')
        for p in x:  
            if int(p) != -1:
                print(i2w[str(p)], end = ', ')       
        print('\n')
        i+=1

AxisError: axis 2 is out of bounds for array of dimension 1

In [98]:
# load the saved best model weights
act_model.load_weights('my_model_weights.h5')
# act_model.load_weights('best_model.hdf5')
 
# predict outputs on validation images
# img = cv2.imread('D:/AMT/batches/aa_piano/validate_x/batch_10/000125088-24_1_1.png', cv2.IMREAD_GRAYSCALE)
# img = np.expand_dims(img, axis=2)
# img = np.expand_dims(img, axis=0)
# print(img.shape)
# # Normalize image
# img = img / 255.
prediction = act_model.predict(valid_img[:5])

# prediction = act_model.predict(img)

# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1], beam_width=500,
                         greedy=True)[0][0])

i2w = json.load(open('C:/Users/trifo/Desktop/AMT/i2w_all.json'))

1/1 [==============================] - 1s 634ms/step


In [99]:
out

array([[ -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
       [356,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1],
       [356,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,
         -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  -1,  

In [100]:
# see the results
i = 0
for x in out:
    print("original_text =  ", end = '')
    for j in valid_padded_txt[i]:
        if j != 0:
            print(i2w[str(j)], end = ', ')
    print('\n')
    print("predicted text = ", end = '')
    for p in x:  
        if int(p) != -1:
            print(i2w[str(int(p))], end = ', ')    
    print('\n')
    i+=1

original_text =  note-Bb5_quarter, note-Eb5_eighth, note-Bb5_eighth, note-C6_eighth, note-Bb5_eighth, note-Ab5_eighth, note-Ab5_eighth, rest-sixteenth, note-Ab5_sixteenth, note-G5_sixteenth, note-Ab5_sixteenth, note-Bb5_sixteenth, note-Ab5_sixteenth, note-G5_sixteenth, note-Ab5_sixteenth, 

predicted text = 

original_text =  multirest-10, note-Bb4_quarter, note-Eb4_eighth, note-Eb5_eighth, note-Eb5_eighth, note-C5_eighth, note-Ab4_eighth, note-Ab4_eighth, rest-quarter, rest-quarter, note-F5_quarter, note-Eb5_eighth., note-C5_sixteenth, note-Bb4_eighth., note-Ab4_sixteenth, note-Ab4_eighth, note-G4_eighth, rest-quarter, 

predicted text = rest-quarter, 

original_text =  multirest-14, rest-quarter, rest-eighth, rest-quarter, note-G4_eighth, note-C5_quarter, rest-eighth, rest-quarter, rest-eighth, rest-quarter, note-C5_eighth, note-E5_quarter, note-C5_eighth, note-A4_quarter, note-A4_eighth, rest-quarter, rest-eighth, 

predicted text = rest-quarter, 

original_text =  multirest-4, rest